In [1]:
!pip3 install -q tensorflow_gpu==2.1.0

In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [3]:
!pip3 install -q ktrain

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id1 = ''

downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('data.csv')  
dataset = pd.read_csv('data.csv')

X = dataset.iloc[:,0].values
y = dataset.iloc[:,1].values

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
x_train
y_train

array(['Internist', 'Pulmonologist', 'Ophthalmologist', ...,
       'Pulmonologist', 'Pulmonologist', 'Internist'], dtype=object)

In [5]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, classes=y_train)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:399: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 8
	95percentile : 14
	99percentile : 17


/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:521: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 7
	95percentile : 15
	99percentile : 17


In [6]:
learner.fit_onecycle(5e-5, 4)



begin training using onecycle policy with max lr of 5e-05...
Train for 615 steps, validate for 39 steps
Epoch 1/4
615/615 [==============================] - 158s 257ms/step - loss: 1.1962 - accuracy: 0.7620 - val_loss: 0.0893 - val_accuracy: 1.0000
Epoch 2/4
615/615 [==============================] - 149s 243ms/step - loss: 0.0421 - accuracy: 0.9997 - val_loss: 0.0077 - val_accuracy: 1.0000
Epoch 3/4
615/615 [==============================] - 149s 242ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 4/4
615/615 [==============================] - 149s 242ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 1.0000


In [7]:
learner.validate(class_names=t.get_classes())

                        precision    recall  f1-score   support

          Cardiologist       1.00      1.00      1.00        67
         Dermatologist       1.00      1.00      1.00       122
       Endocrinologist       1.00      1.00      1.00        77
    Gastroenterologist       1.00      1.00      1.00       117
          Hepatologist       1.00      1.00      1.00       180
   Infectionspecialist       1.00      1.00      1.00        30
             Internist       1.00      1.00      1.00       112
           Neurologist       1.00      1.00      1.00        32
          Neurosurgeon       1.00      1.00      1.00        33
       Ophthalmologist       1.00      1.00      1.00        31
          Orthopaedist       1.00      1.00      1.00        29
 Orthopedic specialist       1.00      1.00      1.00        38
      Otolaryngologist       1.00      1.00      1.00        20
Pediatric Nephrologist       1.00      1.00      1.00        24
          Pediatrician       1.00      

array([[ 67,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  0, 122,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  0,   0,  77,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  0,   0,   0, 117,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 180,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,  30,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 112,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  32,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  33,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,

In [8]:
learner.view_top_losses(n=1, preproc=t)

In [9]:
print(x_test[371])

 joint_pain  neck_pain  knee_pain  hip_joint_pain  swelling_joints  painful_walking


In [10]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [11]:
predictor.predict('itching  nodal_skin_eruptions  dischromic _patches')

'Dermatologist'

In [12]:
# predicted probability scores for each category
predictor.predict_proba('itching  nodal_skin_eruptions  dischromic _patches')

array([0.03162085, 0.131632  , 0.03764983, 0.11796473, 0.05187207,
       0.07863669, 0.04620798, 0.03570081, 0.03464364, 0.03069316,
       0.059517  , 0.0216222 , 0.02545549, 0.06056996, 0.0792785 ,
       0.05732063, 0.06195763, 0.03765684], dtype=float32)

In [13]:
predictor.get_classes()

['Cardiologist',
 'Dermatologist',
 'Endocrinologist',
 'Gastroenterologist',
 'Hepatologist',
 'Infectionspecialist',
 'Internist',
 'Neurologist',
 'Neurosurgeon',
 'Ophthalmologist',
 'Orthopaedist',
 'Orthopedic specialist',
 'Otolaryngologist',
 'Pediatric Nephrologist',
 'Pediatrician',
 'Physician',
 'Pulmonologist',
 'Rheumatologist']

In [14]:
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1

In [ ]:
predictor.explain('itching  nodal_skin_eruptions  dischromic _patches')

In [16]:
predictor.save('/tmp/my_distilbert_predictor')

In [17]:
reloaded_predictor = ktrain.load_predictor('/tmp/my_distilbert_predictor')

In [18]:
reloaded_predictor.predict('itching  skin_rash  dischromic _patches')

'Dermatologist'